In [1]:
import os
import openai
from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
driver = webdriver.Chrome('drivers/chromedriver108')

In [133]:
base = 'https://aws.amazon.com/'

In [134]:
driver.get(base)

In [137]:
doc = bs(driver.find_element(By.TAG_NAME, 'html').get_attribute('innerHTML'), 'html.parser')

In [138]:
title = str(doc.head.title) + '\n'
metas = '\n'.join(list(map(lambda x: str(x), doc.head.find_all('meta'))))
head_string = title + metas

In [139]:
prompt = '''The following text is the url and content of head tag of a website.
url: {0}
head: {1}
The website category is:'''.format(base, head_string)

In [140]:
print(prompt)

The following text is the url and content of head tag of a website.
url: https://aws.amazon.com/
head: <title>Cloud Computing Services - Amazon Web Services (AWS)</title>
<meta content="default-src 'self' data: https://a0.awsstatic.com; connect-src 'self' https://112-tzm-766.mktoresp.com https://112-tzm-766.mktoutil.com https://a0.awsstatic.com https://a0.p.awsstatic.com https://a1.awsstatic.com https://amazonwebservices.d2.sc.omtrdc.net https://amazonwebservicesinc.tt.omtrdc.net https://api.regional-table.region-services.aws.a2z.com https://api.us-west-2.prod.pricing.aws.a2z.com https://auth.aws.amazon.com https://aws.amazon.com https://aws.demdex.net https://b0.p.awsstatic.com https://c0.b0.p.awsstatic.com https://calculator.aws https://chatbot-api.us-east-1.prod.mrc-sunrise.marketing.aws.dev https://cm.everesttech.net https://csml-plc-prod.us-west-2.api.aws/plc/csml/logging https://d0.awsstatic.com https://d1.awsstatic.com https://d1fgizr415o1r6.cloudfront.net https://d2c.aws.amazon

In [141]:
response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    max_tokens=256
)

In [142]:
response

<OpenAIObject text_completion id=cmpl-6fFEApqRcGA1dRtCMnwSTRxLdDb9z at 0x1135f08b0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " Cloud Computing Services"
    }
  ],
  "created": 1675288002,
  "id": "cmpl-6fFEApqRcGA1dRtCMnwSTRxLdDb9z",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 2540,
    "total_tokens": 2543
  }
}